In [1]:
%env PATH=/opt/conda/bin:/opt/spark-2.4.3-bin-hadoop2.7/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/opt/intel_devcloud_support
import os
import sys
sys.path.insert(0, os.path.abspath('/opt/intel_devcloud_support'))
sys.path.insert(0, os.path.abspath('/opt/intel'))

env: PATH=/opt/conda/bin:/opt/spark-2.4.3-bin-hadoop2.7/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/opt/intel_devcloud_support


In [352]:
import videoHtml
videoHtml.videoHTML('Demo', ['/data/demo.mp4'])

In [126]:
%%capture
!python3 /opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name face-detection-adas-binary-0001 --output_dir /data/models
!python3 /opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name head-pose-estimation-adas-0001 --output_dir /data/models
!python3 /opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name landmarks-regression-retail-0009 --output_dir /data/models
!python3 /opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name gaze-estimation-adas-0002 --output_dir /data/models

In [386]:
%%writefile queue_job.sh
#!/bin/bash

exec 1>/output/stdout.log 2>/output/stderr.log

mkdir -p $1
OUTPUT_FILE=$1
DEVICE=$2
PRECISION=$3
INPUT_FILE=$4
THRESHOLD=$5

if [ "$PRECISION" = "FP32" ]; then
    FACEMODELPATH=/data/models/intel/face-detection-adas-binary-0001/FP32-INT1/face-detection-adas-binary-0001.xml
    POSEMODELPATH=/data/models/intel/head-pose-estimation-adas-0001/FP32/head-pose-estimation-adas-0001.xml
    LANDMARKSMODELPATH=/data/models/intel/landmarks-regression-retail-0009/FP32/landmarks-regression-retail-0009.xml    
    GAZEMODELPATH=/data/models/intel/gaze-estimation-adas-0002/FP32/gaze-estimation-adas-0002.xml        
else
    FACEMODELPATH=/data/models/intel/face-detection-adas-binary-0001/FP32-INT1/face-detection-adas-binary-0001.xml
    POSEMODELPATH=/data/models/intel/head-pose-estimation-adas-0001/FP16/head-pose-estimation-adas-0001.xml
    LANDMARKSMODELPATH=/data/models/intel/landmarks-regression-retail-0009/FP16/landmarks-regression-retail-0009.xml    
    GAZEMODELPATH=/data/models/intel/gaze-estimation-adas-0002/FP16/gaze-estimation-adas-0002.xml        
fi

if echo "$DEVICE" | grep -q "FPGA"; then # if device passed in is FPGA, load bitstream to program FPGA
    #Environment variables and compilation for edge compute nodes with FPGAs
    source /opt/intel/init_openvino.sh
    aocl program acl0 /opt/intel/openvino/bitstreams/a10_vision_design_sg1_bitstreams/2019R4_PL1_FP16_MobileNet_Clamp.aocx
fi

ls /data/*

python3 main.py -fm ${FACEMODELPATH} \
                -pm ${POSEMODELPATH} \
                -lm ${LANDMARKSMODELPATH} \
                -gm ${GAZEMODELPATH} \
                -i ${INPUT_FILE} \
                -o ${OUTPUT_FILE} \
                -d ${DEVICE} \
                -c ${THRESHOLD}

cd /output

tar zcvf output.tgz *

Overwriting queue_job.sh


In [404]:
device = "CPU"
output_path = "/output/results/controller"
precision = "FP32"
thres=0.5
input_file = "demo.mp4"
params = ' {} {} {} {} {}'.format(output_path, device, precision, input_file, thres)


In [410]:
job_id = !qsub queue_job.sh -l nodes=1:tank-870:i5-6500te -d . -F "{params}" -N "cpu" -wd .

In [411]:
job_id[0]

'BATtMXtL9lAg616irAoYSgUPIW8dhQPq'

In [412]:
import liveQStat
liveQStat.liveQStat()

In [413]:
import get_results
get_results.getResults(job_id[0], filename='output.tgz', blocking=True)


getResults() is blocking until results of the job (id:BATtMXtL9lAg616irAoYSgUPIW8dhQPq) are ready.
Please wait.......Success!
output.tgz was downloaded in the same folder as this notebook.


In [414]:
!tar zxf output.tgz
!cat stderr.log

Traceback (most recent call last):
  File "main.py", line 266, in <module>
    main()
  File "main.py", line 77, in main
    assert os.path.isfile(args.input), "Specified input file doesn't exist"
AssertionError: Specified input file doesn't exist


In [415]:
!cat stdout.log


/data/fetch_model.ipynb
/data/videofile.mp4

/data/legacy_ignore:
data

/data/models:
intel

/data/queue_param:
manufacturing.npy
retail.npy
transportation.npy

/data/resources:
car.png
manufacturing.mp4
retail.mp4
transportation.mp4
Arg input demo.mp4
results/
results/controller/
stderr.log


In [416]:
os.environ["VIDEO"] = "bin/demo.mp4"
!ln -sf ./bin/demo.mp4